# Projet final Adrien Roux

In [25]:
from shutil import copyfileobj
from urllib import request
import requests
from bs4 import BeautifulSoup
import pandas as pd
import uuid
import xlwings as xw
import pickle
import re
import numpy as np
from sklearn.linear_model import LinearRegression

## Téléchargement des classements au format excel

In [2]:
#on crée la liste des urls des fichiers excels
url_root = "https://www.vendeeglobe.org/download-race-data/vendeeglobe_"
url_ls = []

month_ls = ["11","12","01","02","03"]
hour_ls = ["_040000","_080000","_110000","_140000","_170000","_210000"]

for month in month_ls:
    for day in range(31):
        day += 1
        if month == "11":
            if day < 9:
                day = 42
            elif day == 31:
                day = 42
        elif month == "02":
            if day > 28:
                day = 42
        elif month == "03":
            if day > 4:
                day = 42
        if day == 42:
            pass
        else:
            if month == "11" or month == "12":
                year_str = "2020"
            else:
                year_str = "2021"
            if day < 10:
                day_str = "0" + str(day)
            else:
                day_str = str(day)
            for hour in hour_ls:
                url = url_root + year_str + month + day_str + hour + ".xlsx"
                url_ls.append(url)

In [3]:
url_ls

['https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201109_040000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201109_080000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201109_110000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201109_140000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201109_170000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201109_210000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201110_040000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201110_080000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201110_110000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201110_140000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201110_170000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201110_210000

In [4]:
url_ls_first = ['https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201108_120200.xlsx','https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201108_140000.xlsx','https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201108_150000.xlsx','https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201108_170000.xlsx','https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201108_210000.xlsx']
url_ls_last = ['https://www.vendeeglobe.org/download-race-data/vendeeglobe_20210305_040000.xlsx','https://www.vendeeglobe.org/download-race-data/vendeeglobe_20210305_080000.xlsx']
url_ls = url_ls_first + url_ls + url_ls_last

In [5]:
url_ls

['https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201108_120200.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201108_140000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201108_150000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201108_170000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201108_210000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201109_040000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201109_080000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201109_110000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201109_140000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201109_170000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201109_210000.xlsx',
 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201110_040000

In [6]:
url_ls[0][-32:-5]

'vendeeglobe_20201108_120200'

In [45]:
#on utilise cette liste d'url pour télécharger tous les classements
for url in url_ls:
    filename = "../kit/" + url[-32:]
    with request.urlopen(url) as response, open(filename, 'wb') as out_file:
        copyfileobj(response, out_file)

## Rendre les fichiers excels lisibles

In [11]:
#fonction présente sur github permettant de reformater les fichiers excels afin qu'ils soient lisibles par pandas
def save_with_xlwings(file, name):
    tempfile = f'../kit_processed/' + name + '.xlsx'
    excel_app = xw.App(visible=False)
    excel_book = excel_app.books.open(file)
    excel_book.save(tempfile)
    excel_book.close()
    excel_app.quit()
    return tempfile

In [12]:
#formatage des fichiers
for url in url_ls:
    filename = "../kit/" + url[-32:]
    save_with_xlwings(filename, url[-32:-5])

In [31]:
#exemple du traitement utilisé pour tous les fichiers jusqu'à leur changement de format en fin de course
df = pd.read_excel("../kit_processed/vendeeglobe_20201108_140000.xlsx", header = 4)
df = df.drop(['Unnamed: 0', 'Cap\nHeading', 'Vitesse\nSpeed', 'VMG\nVMG', 'Distance\nDistance'], axis=1)
df = df.drop(['Cap\nHeading.2', 'Vitesse\nSpeed.2', 'VMG\nVMG.2', 'Distance\nDistance.2', "Heure FR\nHour FR"], axis=1)
df = df.rename(columns={"Unnamed: 1": "rang","Unnamed: 2": "Numero","Unnamed: 3": "Nom","Latitude\nLatitude": "Latitude","Longitude\nLongitude": "Longitude","Cap\nHeading.1": "Cap","Vitesse\nSpeed.1": "Vitesse","VMG\nVMG.1": "VMG","Distance\nDistance.1": "Distance","Unnamed: 19": "DTF","Unnamed: 20": "DTL"})
df = df.drop([33,34,35,36])
df['Numero'] = df['Numero'].apply(lambda x: x.split('\n')[1])
df

,rang,Numero,Nom,Latitude,Longitude,Cap,Vitesse,VMG,Distance,DTF,DTL
0,1,FRA 18,Louis Burton\nBureau Vallée 2,46°24.46'N,01°50.48'W,357°,0.0 kts,0.0 kts,2788.0 nm,24293.9 nm,0.0 nm
1,2,MON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,46°24.34'N,01°49.82'W,357°,0.0 kts,0.0 kts,2787.9 nm,24294.2 nm,0.4 nm
2,3,FRA 8,Jérémie Beyou\nCharal,46°24.91'N,01°49.99'W,357°,0.0 kts,0.0 kts,2788.5 nm,24294.3 nm,0.5 nm
3,4,FRA 59,Thomas Ruyant\nLinkedOut,46°24.71'N,01°49.68'W,357°,0.0 kts,0.0 kts,2788.3 nm,24294.5 nm,0.6 nm
4,5,FRA 53,Maxime Sorel\nV And B Mayenne,46°24.59'N,01°49.56'W,357°,0.0 kts,0.0 kts,2788.1 nm,24294.5 nm,0.6 nm
5,6,FRA 56,Fabrice Amedeo\nNewrest - Art et Fenetres,46°25.19'N,01°49.87'W,357°,0.0 kts,0.0 kts,2788.7 nm,24294.5 nm,0.7 nm
6,7,FRA 01,Jean Le Cam\nYes we Cam !,46°24.90'N,01°49.49'W,357°,0.0 kts,0.0 kts,2788.4 nm,24294.7 nm,0.8 nm
7,8,GBR 99,Alex Thomson\nHugo Boss,46°25.21'N,01°49.45'W,357°,0.0 kts,0.0 kts,2788.7 nm,24294.8 nm,1.0 nm
8,9,FRA 69,Sébastien Destremau\nMerci,46°25.04'N,01°49.32'W,357°,0.0 kts,0.0 kts,2788.6 nm,24294.8 nm,1.0 nm
9,10,GBR 777,Pip Hare\nMedallia,46°25.22'N,01°49.36'W,357°,0.0 kts,0.0 kts,2788.8 nm,24294.9 nm,1.0 nm


In [34]:
#traitement des classements et sauvegarde du résultat sous forme de dataframe avec pickle
for url in url_ls:
    if url[-32:] == 'vendeeglobe_20201108_120200.xlsx':
        pass
    elif url[-32:] == 'vendeeglobe_20210127_140000.xlsx':
        break
    else:
        filename = "../kit_processed/" + url[-32:]
        df = pd.read_excel(filename, header = 4)
        df = df.drop(['Unnamed: 0', 'Cap\nHeading', 'Vitesse\nSpeed', 'VMG\nVMG', 'Distance\nDistance'], axis=1)
        df = df.drop(['Cap\nHeading.2', 'Vitesse\nSpeed.2', 'VMG\nVMG.2', 'Distance\nDistance.2', "Heure FR\nHour FR"], axis=1)
        df = df.rename(columns={"Unnamed: 1": "rang","Unnamed: 2": "Numero","Unnamed: 3": "Nom","Latitude\nLatitude": "Latitude","Longitude\nLongitude": "Longitude","Cap\nHeading.1": "Cap","Vitesse\nSpeed.1": "Vitesse","VMG\nVMG.1": "VMG","Distance\nDistance.1": "Distance","Unnamed: 19": "DTF","Unnamed: 20": "DTL"})
        df = df.drop([33,34,35,36])
        df['Numero'] = df['Numero'].apply(lambda x: x.split('\n')[1])
        pickle.dump( df, open( "../kit_pickled/" + url[-32:-5] + ".p", "wb" ) )

## Scrapping de la fiche technique des voiliers

In [7]:
#on utilise requests et beautifulsoup afin de scraper la page avec les caractéristiques des voiliers
r1 = requests.get("https://www.vendeeglobe.org/fr/glossaire")
content1 = r1.content.decode('utf-8')
soup1 = BeautifulSoup(content1)
fiche_ls = soup1.find_all('div', {'class': "boats-list__popup-infos"})

In [8]:
fiche_ls

[<div class="boats-list__popup-infos">
 <h3 class="boats-list__popup-title">NEWREST - ART &amp; FENÊTRES</h3>
 <ul class="boats-list__popup-specs-list">
 <li>Numéro de voile : FRA 56</li>
 <li>Anciens noms du bateau : No Way Back, Vento di Sardegna</li>
 <li>Architecte : VPLP/Verdier</li>
 <li>Chantier : Persico Marine</li>
 <li>Date de lancement : 01 Août 2015</li>
 <li>Longueur : 18,28 m</li>
 <li>Largeur : 5,85 m</li>
 <li>Tirant d'eau : 4,50 m</li>
 <li>Déplacement (poids) : 7 t</li>
 <li>Nombre de dérives : foils</li>
 <li>Hauteur mât : 29 m</li>
 <li>Voile quille : monotype</li>
 <li>Surface de voiles au près : 320 m2</li>
 <li>Surface de voiles au portant : 570 m2</li>
 </ul>
 <div class="boats-list__popup-skipper-line">
 <a class="boats-list__popup-skipper-link" href="/fr/skippers/93/fabrice-amedeo">
 <span>Consulter la fiche skipper</span>
 </a>
 </div>
 <button class="mfp-close m--desktop"><svg class="icon icon-close" role="img"><use xlink:href="/public/images/web1/pictos/ico

In [9]:
#on traite le scrap html afin d'extraire les caractéristiques de chaque bateau sous forme d'un dictionnaire avec son nom comme clé
boat_dict = {}
for boat in fiche_ls:
    boat_dict[boat.find('h3', {'class': "boats-list__popup-title"}).text] = {}
    boat_spec_ls = boat.find_all('li')
    for spec in boat_spec_ls:
        spec = str(spec)[4:-5]
        spec_split = spec.split(" : ")
        boat_dict[boat.find('h3', {'class': "boats-list__popup-title"}).text][spec_split[0]] = spec_split[1]

In [10]:
boat_dict

{'NEWREST - ART & FENÊTRES': {'Numéro de voile': 'FRA 56',
  'Anciens noms du bateau': 'No Way Back, Vento di Sardegna',
  'Architecte': 'VPLP/Verdier',
  'Chantier': 'Persico Marine',
  'Date de lancement': '01 Août 2015',
  'Longueur': '18,28 m',
  'Largeur': '5,85 m',
  "Tirant d'eau": '4,50 m',
  'Déplacement (poids)': '7 t',
  'Nombre de dérives': 'foils',
  'Hauteur mât': '29 m',
  'Voile quille': 'monotype',
  'Surface de voiles au près': '320 m2',
  'Surface de voiles au portant': '570 m2'},
 'PURE - Best Western®': {'Numéro de voile': 'FRA 49',
  'Anciens noms du bateau': 'Gitana Eighty, Synerciel, Newrest-Matmut',
  'Architecte': 'Bruce Farr Design',
  'Chantier': 'Southern Ocean Marine (Nouvelle Zélande)',
  'Date de lancement': '08 Mars 2007',
  'Longueur': '18,28m',
  'Largeur': '5,80m',
  "Tirant d'eau": '4,50m',
  'Déplacement (poids)': '9t',
  'Nombre de dérives': '2',
  'Hauteur mât': '28m',
  'Voile quille': 'acier forgé',
  'Surface de voiles au près': '280 m2',
  'S

In [11]:
#Comme l'écriture du nom des bateaux n'est pas la meme entre la fiche technique et les classements,
#On fait un second dictionnaire utilisant le numéro de voile des bateaux comme clé
#Le numéro de voile change également en fonction des formats et on utilise donc le formalisme utilisé dans
#Les classements afin de pouvoir lier les deux sources de données
boat_dict_tech = {}
for key, value in boat_dict.items():
    if key == '':
        pass
    else:
        if key == 'LinkedOut':
            numero_voile = '59'
        else:
            numero_voile = re.sub("[^0-9]", "", value['Numéro de voile'])
            if numero_voile == '001':
                numero_voile = '01'
            elif numero_voile == '16':
                numero_voile = '10'
            elif numero_voile == '2':
                numero_voile = '02'
            elif numero_voile == '08':
                numero_voile = '8'
            elif numero_voile == '07':
                numero_voile = '7'
            elif numero_voile == '77':
                numero_voile = '777'
            value.pop('Numéro de voile')
        boat_dict_tech[numero_voile] = value
    
boat_dict_tech

{'56': {'Anciens noms du bateau': 'No Way Back, Vento di Sardegna',
  'Architecte': 'VPLP/Verdier',
  'Chantier': 'Persico Marine',
  'Date de lancement': '01 Août 2015',
  'Longueur': '18,28 m',
  'Largeur': '5,85 m',
  "Tirant d'eau": '4,50 m',
  'Déplacement (poids)': '7 t',
  'Nombre de dérives': 'foils',
  'Hauteur mât': '29 m',
  'Voile quille': 'monotype',
  'Surface de voiles au près': '320 m2',
  'Surface de voiles au portant': '570 m2'},
 '49': {'Anciens noms du bateau': 'Gitana Eighty, Synerciel, Newrest-Matmut',
  'Architecte': 'Bruce Farr Design',
  'Chantier': 'Southern Ocean Marine (Nouvelle Zélande)',
  'Date de lancement': '08 Mars 2007',
  'Longueur': '18,28m',
  'Largeur': '5,80m',
  "Tirant d'eau": '4,50m',
  'Déplacement (poids)': '9t',
  'Nombre de dérives': '2',
  'Hauteur mât': '28m',
  'Voile quille': 'acier forgé',
  'Surface de voiles au près': '280 m2',
  'Surface de voiles au portant': '560 m2'},
 '72': {'Anciens noms du bateau': 'Famille Mary-Etamine du Ly

## Faut-il un foil pour gagner le Vendée Globe?

Pour répondre à cette question, on s'intéresse au classement final des bateaux. Ce fichier excel est dans un format différent de ceux traités précedemment. Il doit donc être traité séparemment. On y ajout également la colonne foil qui prend pour valeur "oui" si le bateau présente un foil et "non" autrement. Cette information est issue de la fiche technique des voiliers scrappée dans la partie précédente.

In [12]:
classement_final_ex = "../kit_processed/vendeeglobe_20210305_080000.xlsx"
classement_final = pd.read_excel(classement_final_ex, header = 4)
classement_final = classement_final[['Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 13','Vitesse\nSpeed.1']]
classement_final = classement_final.rename(columns={"Unnamed: 1": "Rang","Unnamed: 2": "Numero","Unnamed: 3": "Nom","Unnamed: 13": "TTL","Vitesse\nSpeed.1": "Vitesse"})
classement_final = classement_final.drop([33,34,35,36])
classement_final['Numero'] = classement_final['Numero'].apply(lambda x: x.split('\n')[1])
classement_final['Rang'] = classement_final['Rang'].apply(lambda x: x.split('\n')[0] if x != 'RET' else x)
classement_final['Foil'] = classement_final['Numero'].apply(lambda x: 'oui' if boat_dict_tech[re.sub("[^0-9]", "",x)]['Nombre de dérives'] == 'foils' or boat_dict_tech[re.sub("[^0-9]", "",x)]['Nombre de dérives'] == 'foiler' else 'non')
classement_final

,Rang,Numero,Nom,TTL,Vitesse,Foil
0,1,FRA 17,Yannick Bestaven\nMaître Coq IV,NaN,14.8 kts,oui
1,2,FRA 79,Charlie Dalin\nAPIVIA,02h 31min 01s,15.1 kts,oui
2,3,FRA 18,Louis Burton\nBureau Vallée 2,06h 40min 26s,14.8 kts,oui
3,4,FRA 01,Jean Le Cam\nYes we Cam !,10h 00min 09s,14.1 kts,non
4,5,MON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,11h 14min 59s,14.7 kts,oui
5,6,FRA 59,Thomas Ruyant\nLinkedOut,11h 37min 15s,15.1 kts,oui
6,7,FRA 1000,Damien Seguin\nGroupe APICIL,18h 13min 34s,14.2 kts,non
7,8,ITA 34,Giancarlo Pedote\nPrysmian Group,18h 57min 34s,14.7 kts,oui
8,9,FRA 09,Benjamin Dutreux\nOMIA - Water Family,1j 16h 00min 34s,14.2 kts,non
9,10,FRA 53,Maxime Sorel\nV And B Mayenne,2j 10h 45min 29s,13.8 kts,non


In [31]:
print("La proportion de foils sur l'ensemble des bateaux est de {:.0f}%".format(sum(classement_final['Foil'].apply(lambda x: 1 if x=='oui' else 0))/len(classement_final)*100))

La proportion de foils sur l'ensemble des bateaux est de 58%


De plus, la proportion de bateaux dans le top 5 avec foil est de 80%, cela prouve l'efficacité de cette technologie.

In [17]:
mean_speed_foil = 0
i = 0
for index, row in classement_final.iterrows():
    if index == 25:
        break
    if row['Foil'] == 'oui':
        mean_speed_foil += float(re.sub(" kts", "", row['Vitesse']))
        i +=1
mean_speed_foil = mean_speed_foil / i

mean_speed_no_foil = 0
i = 0
for index, row in classement_final.iterrows():
    if index == 25:
        break
    if row['Foil'] == 'non':
        mean_speed_no_foil += float(re.sub(" kts", "", row['Vitesse']))
        i +=1
mean_speed_no_foil = mean_speed_no_foil / i

print("La vitesse moyenne des bateaux avec foil est de {:.2f} noeuds tandis que celle des bateaux sans foil est de {:.2f} noeuds.".format(mean_speed_foil,mean_speed_no_foil))

La vitesse moyenne des bateaux avec foil est de 13.93 noeuds tandis que celle des bateaux sans foil est de 12.48 noeuds.


En revanche, on constate que tous les bateaux qui ont abandonné sauf un ont un foil. Cela s'explique par le fait que les foils sont constitués de lames de carbone qui sont plus fragiles qu'une dérive standard. "Qui ose gagne".

On peut également s'intéresser à l'age des bateaux en course.

In [18]:
classement_final['Date de lancement'] = classement_final['Numero'].apply(lambda x: boat_dict_tech[re.sub("[^0-9]", "",x)]['Date de lancement'])
classement_final

,Rang,Numero,Nom,TTL,Vitesse,Foil,Date de lancement
0,1,FRA 17,Yannick Bestaven\nMaître Coq IV,NaN,14.8 kts,oui,12 Mars 2015
1,2,FRA 79,Charlie Dalin\nAPIVIA,02h 31min 01s,15.1 kts,oui,05 Août 2019
2,3,FRA 18,Louis Burton\nBureau Vallée 2,06h 40min 26s,14.8 kts,oui,09 Juin 2015
3,4,FRA 01,Jean Le Cam\nYes we Cam !,10h 00min 09s,14.1 kts,non,03 Janvier 2007
4,5,MON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,11h 14min 59s,14.7 kts,oui,07 Août 2015
5,6,FRA 59,Thomas Ruyant\nLinkedOut,11h 37min 15s,15.1 kts,oui,03 Septembre 2019
6,7,FRA 1000,Damien Seguin\nGroupe APICIL,18h 13min 34s,14.2 kts,non,10 Août 2008
7,8,ITA 34,Giancarlo Pedote\nPrysmian Group,18h 57min 34s,14.7 kts,oui,02 Avril 2015
8,9,FRA 09,Benjamin Dutreux\nOMIA - Water Family,1j 16h 00min 34s,14.2 kts,non,03 Juillet 2007
9,10,FRA 53,Maxime Sorel\nV And B Mayenne,2j 10h 45min 29s,13.8 kts,non,07 Septembre 2007


On remarque en ajoutant la date de lancement des bateaux que  bateaux les plus modernes ont tendance à mieux performer que la moyenne ce qui s'explique aussi par le fait que tous les bateaux plus récents que 2015 ont des foils.

## Régression linéaire VMG-rang

Dans cette partie, on réalise une régression linéaire des du rang en fonction de la vitesse moyenne utile des bateaux afin de voir si cette mesure est un bon estimateur de la performance des bateaux.

Pour réaliser cette régression, on utilise la méthode des moindres carrés ordinaire implémentée dans la librairie scikit learn avec la vitesse utile moyenne des voiliers comme variable explicative et le rang comme variable à expliquer.

In [24]:
X = classement_final['Vitesse'][0:25].apply(lambda x: float(re.sub(" kts", "", x)))
y = classement_final['Rang'][0:25].apply(lambda x: float(x))

In [27]:
reg = LinearRegression().fit(X.to_numpy().reshape(-1, 1), y)

In [30]:
print("Le r2 de la regression est de {:.2f}".format(reg.score(X.to_numpy().reshape(-1, 1), y)))

Le r2 de la regression est de 0.93


L'estimation du rang du bateau en fonction de sa vitesse utile moyenne en utilisant la méthode des moindres carrés nous permet d'obtenir un coefficient de determination 0.93. Cela signifie que la vitesse utile moyenne explique 93% de la variance du rang est expliquée par la variance de la VMG et que ces deux grandeurs sont fortement corrélées.

In [33]:
print("Le coefficient de la pente de regression est de {}".format(reg.coef_))

Le coefficient de la pente de regression est de [-5.02305113]


Ce coefficient est négative ce qui implique que plus un bateau est rapide, le meilleur est son rang.